<a href="https://colab.research.google.com/github/farhanwadia/EEGClassification/blob/Farhan/EEG_Classification_Model_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EEG Classification Model Testing

## 1. Clone the GitHub Repository

In [1]:
!git clone https://github.com/farhanwadia/EEGClassification
!cd EEGClassification && git checkout Farhan

#Change the working directory to the EEGClassification folder 
%cd EEGClassification
!pwd

Cloning into 'EEGClassification'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 10618 (delta 28), reused 24 (delta 8), pack-reused 10538
Receiving objects: 100% (10618/10618), 4.00 GiB | 23.46 MiB/s, done.
Resolving deltas: 100% (538/538), done.
Checking out files: 100% (10468/10468), done.
Branch 'Farhan' set up to track remote branch 'Farhan' from 'origin'.
Switched to a new branch 'Farhan'
/content/EEGClassification
/content/EEGClassification


### Imports

In [2]:
#!pip install mne

import numpy as np
import os
#import mne
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import pickle

import csv
import json
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim

## 2. Split data into training, validation, and test sets

In [3]:
def loadCSV(filename):
    # Takes in a string of the csv file name where the EEG data is (# of measurements by 4 channels)
    # Returns the data as an np array, transposed, with all values /10**6
    data = np.loadtxt(filename, delimiter=',')
    data = data.T / 10**6
    return data

def jsonToNp(pathToJSON):
    with open(pathToJSON, "r") as f:
        data = np.array(json.load(f))
    return data 

def getFiles(parentPath):
    # Returns all files in the folder and its subfolders as a list
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(parentPath):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]
    return listOfFiles

def createDataset(openFiles, closedFiles):
    n = len(openFiles) + len(closedFiles)
    w = loadCSV(openFiles[0]).shape[0]
    h = loadCSV(openFiles[0]).shape[1]

    X = np.zeros((n, w, h))
    y = np.zeros((n))
    filenames = []

    # Put files into np arrays. 
    #Assumes all data has already been cut to correct lengths, with no inconsistincies in the numper of data points between files
    i = 0
    for lst in [openFiles, closedFiles]:
        for file in lst:
            data = loadCSV(file)
            X[i] = data
            if lst == openFiles:
                y[i] = 1            #Label 1: Open
            else:
                y[i] = 0            #Label 0: Closed

            name = file.split(r"\\")[-1].split(".")[0]
            filenames.append(name)
            i = i+1
    return X, y, filenames

def createDatasetFromJSON(openFiles, closedFiles):
    n = len(openFiles) + len(closedFiles)
    c = jsonToNp(openFiles[0]).shape[0]
    h = jsonToNp(openFiles[0]).shape[1]
    w = jsonToNp(openFiles[0]).shape[2]

    X = np.zeros((n, c, h, w))
    y = np.zeros((n))
    filenames = []
    
    # Put files into np arrays.
    i = 0
    for lst in [openFiles, closedFiles]:
        for file in lst:
            data = jsonToNp(file)
            X[i] = data
            if lst == openFiles:
                y[i] = 1            #Label 1: Open
            else:
                y[i] = 0            #Label 0: Closed

            name = file.split(r"\\")[-1].split(".")[0]
            filenames.append(name)
            i = i+1
    return X, y, filenames


def shuffleDataset(X, y, filenames):
    indices = np.arange(X.shape[0])
    seed = 21 
    np.random.seed(seed)
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    filenames = np.array(filenames)
    filenames = filenames[indices]
    return X, y, filenames

def splitDataset(X, y, filenames, percent_train):
    n = X.shape[0]
    X_train = X[:int(percent_train*n)]
    X_val = X[int(percent_train*n):]
    y_train = y[:int(percent_train*n)]
    y_val = y[int(percent_train*n):]
    filenames_train = filenames[:int(percent_train*n)]
    filenames_val = filenames[int(percent_train*n):]
    return X_train, y_train, filenames_train, X_val, y_val, filenames_val

### Time Series

In [4]:
#Get the dataset split exactly as was from the training notebook
#Need to do it this way because of reproducibility issues with np.random.seed (see here https://towardsdatascience.com/stop-using-numpy-random-seed-581a9972805f)
filenames_train = ['/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T5 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T38 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T9 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T38 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T12 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ2_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T39 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T16 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T35 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T75 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T26 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ2_T1 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T18 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP2_T4 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T27 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ2_T3 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T13 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T14 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T75 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T40 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T14 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T13 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T21 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T20 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T55 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T25 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T57 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T11 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T38 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T32 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T37 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ2_T9 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T4 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T30 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T70 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T22 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T46 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T36 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T39 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T35 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T24 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T24 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T34 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T84 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T17 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T65 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T1 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T28 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T5 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T12 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T28 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T20 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T20 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T45 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T70 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T35 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T70 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T36 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T19 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T13 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T17 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T58 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T20 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T56 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T32 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T30 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T29 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T48 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T9 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T62 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T13 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T7 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T51 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T12 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T7 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T14 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T24 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T29 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ2_T6 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T60 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T23 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T76 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T23 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T25 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T38 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T43 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T8 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T31 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T37 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T6 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T1 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T38 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T17 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T10 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T22 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T32 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T1 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T24 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T40 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T11 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T24 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T14 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T43 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T23 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T44 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T2 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T18 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T39 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T35 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T34 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T26 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T3 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T6 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T5 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T12 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T71 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T6 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T16 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T3 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T52 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T31 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T17 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T6 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T14 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T9 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T25 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP2_T8 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T8 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T25 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T25 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T19 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T13 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T11 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T21 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T17 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T26 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T30 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T13 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T28 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T8 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T71 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T6 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T39 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T26 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T9 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T15 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T66 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T44 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T11 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T28 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T50 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T26 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T23 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T3 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T8 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T14 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T12 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ2_T11 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T24 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T33 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T5 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T9 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T38 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T81 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T13 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T20 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T65 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T10 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T37 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T25 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T17 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ2_T8 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T31 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T7 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T3 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T18 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T19 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T13 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T5 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T14 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T19 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T7 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T18 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T10 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T8 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ2_T4 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ2_T7 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T1 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T7 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T22 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T12 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T61 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T28 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T18 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T40 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T39 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T22 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T9 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T4 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T24 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T10 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T29 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T41 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T7 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T21 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T64 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T29 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T51 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T8 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T24 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T55 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T26 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T22 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T31 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T23 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T23 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T1 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T17 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T1 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T58 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T12 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T54 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T9 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T29 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T3 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T18 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T53 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T29 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T14 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T27 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T32 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T26 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T30 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T28 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T24 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T31 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T26 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T22 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T19 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T20 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T3 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T8 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T53 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T26 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T18 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T19 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T22 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ2_T12 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T34 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T16 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T32 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T39 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T13 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T35 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T2 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T1 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T26 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T17 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T1 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T33 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T87 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T21 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T77 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T1 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T28 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T36 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T31 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T1 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T11 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T10 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T8 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T25 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T3 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T7 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T39 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T13 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T28 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T35 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T21 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T10 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T43 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T27 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T13 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T25 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T32 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T40 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T14 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T27 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T6 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T30 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T13 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T15 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T16 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T33 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T2 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T5 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T40 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T14 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T22 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP2_T6 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T23 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T39 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T12 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T22 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T35 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T27 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T34 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T14 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T9 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T27 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T21 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP2_T14 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T38 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T11 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T26 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T16 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T32 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T6 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T59 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T13 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T18 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T12 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T6 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T34 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T8 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T33 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T6 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T32 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T78 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T7 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T10 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T20 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T5 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T40 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T7 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T12 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T8 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T31 Open (valve) - Filtered.csv']

filenames_val = ['/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T24 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T32 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T15 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T16 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T17 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T34 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T34 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T3 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T30 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T22 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T27 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T33 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T37 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T10 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T33 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T34 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T12 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T27 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T16 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T41 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T10 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T57 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T28 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T4 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T4 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP2_T18 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T21 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T4 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T19 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T30 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T10 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T1 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T38 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T4 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T17 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T12 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T4 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T5 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T80 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T3 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T7 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T30 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T52 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T40 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T12 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T23 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T29 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T22 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T3 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T78 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T50 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T62 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T33 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T47 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T29 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T23 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T7 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T27 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T31 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T37 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T61 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T23 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T9 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T61 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T6 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T1 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ2_T5 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T35 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T1 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T34 Close (valve) - Filtered.csv']

filenames_test = ['/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T36 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T17 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T15 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T39 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T30 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T35 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T75 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T20 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T16 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T21 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T20 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T11 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP2_T2 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T16 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T11 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T7 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ2_T10 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T32 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T4 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T36 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T29 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T73 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T26 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T22 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T78 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T36 Close (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T20 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T77 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T22 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T26 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T36 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T76 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T37 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T85 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T46 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T18 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW2_T39 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP2_T12 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T69 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T32 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T18 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T17 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T9 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T11 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T19 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T5 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T30 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T11 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T15 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T17 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T4 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T13 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T32 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T11 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T31 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T2 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T5 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T43 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T33 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T10 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T24 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T27 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//SD_T14 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T31 Open (valve)_(2) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//AZ_T5 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T17 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T25 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T18 Close (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T17 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T40 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T19 Open (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T53 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T7 Close (valve) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T15 Open (valve)_(1) - Filtered.csv',
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T4 Close (valve) - Filtered.csv']

In [5]:
openPath = os.getcwd() + r"//Nov 2020 Filtered Data//5s//OPEN//"
closedPath = os.getcwd() + r"//Nov 2020 Filtered Data//5s//CLOSE//"
openPath2 = os.getcwd() + r"//Feb 2021 Filtered Data//5s//OPEN//"
closedPath2 = os.getcwd() + r"//Feb 2021 Filtered Data//5s//CLOSE//"
openFiles = getFiles(openPath) + getFiles(openPath2)
closedFiles = getFiles(closedPath) + getFiles(closedPath2) 

open = [i for i in filenames_train if "OPEN" in i]
close = [i for i in filenames_train if "CLOSE" in i]
X_train, y_train, filenames_train = createDataset(open, close)

open = [i for i in filenames_val if "OPEN" in i]
close = [i for i in filenames_val if "CLOSE" in i]
X_val, y_val, filenames_val = createDataset(open, close)

open = [i for i in filenames_test if "OPEN" in i]
close = [i for i in filenames_test if "CLOSE" in i]
X_test, y_test, filenames_test = createDataset(open, close)

#X, y, filenames = shuffleDataset(X, y, filenames)
#X_train, y_train, filenames_train, X_other, y_other, filenames_other = splitDataset(X, y, filenames, percent_train=0.7)
#X_val, y_val, filenames_val, X_test, y_test, filenames_test = splitDataset(X_other, y_other, filenames_other, percent_train=0.5) 

print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)
print("X_val Shape: ", X_val.shape)
print("y_val Shape: ", y_val.shape)

X_train Shape:  (350, 4, 1280)
y_train Shape:  (350,)
X_val Shape:  (75, 4, 1280)
y_val Shape:  (75,)


Print the filenames for the datasets (can verify with training notebook that all sets are the same and that test data wasn't used for training)

In [6]:
for f in filenames_train:
    print(f)

/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T5 Open (valve)_(1) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T39 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T35 Open (valve)_(2) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T75 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ2_T1 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T27 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ2_T3 Open (valve) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T15 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T13 Open (valve)_(1) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T14 Open (valve) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OP

In [7]:
for f in filenames_val:
    print(f)

/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T15 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T17 Open (valve)_(2) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T34 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T3 Open (valve) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T30 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T33 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T37 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T33 Open (valve)_(1) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T27 Open (valve) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T57 Open (valve) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//

In [8]:
for f in filenames_test:
    print(f)

/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T17 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T15 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T39 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T35 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T21 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T11 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP2_T7 Open (valve)_(1) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T29 Open (valve)_(1) - Filtered
/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//CJ_T73 Open (valve) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T37 Open (valve)_(1) - Filtered
/content/EEGClassification//Feb 2021 Filtered Data//5s//O

### Time-Frequency (Wavelets Transformed)

In [9]:
#openPath = os.getcwd() + r"//Nov 2020 Filtered Data//5s//Spectograms//OPEN//"
#closedPath = os.getcwd() + r"//Nov 2020 Filtered Data//5s//Spectograms//CLOSE//"
#openPath2 = os.getcwd() + r"//Feb 2021 Filtered Data//5s//Spectograms//OPEN//"
#closedPath2 = os.getcwd() + r"//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//"
#openFiles = getFiles(openPath) + getFiles(openPath2)
#closedFiles = getFiles(closedPath) + getFiles(closedPath2) 

#X, y, filenames = createDatasetFromJSON(openFiles, closedFiles)
#X, y, filenames = shuffleDataset(X, y, filenames)
#X_train_wv, y_train_wv, filenames_train_wv, X_other, y_other, filenames_other = splitDataset(X, y, filenames, percent_train=0.7)
#X_val_wv, y_val_wv, filenames_val_wv, X_test_wv, y_test_wv, filenames_test_wv = splitDataset(X_other, y_other, filenames_other, percent_train=0.5) 

#print("X_train_wv Shape: ", X_train_wv.shape)
#print("y_train_wv Shape: ", y_train_wv.shape)
#print("X_val_wv Shape: ", X_val_wv.shape)
#print("y_val_wv Shape: ", y_val_wv.shape)

Print the filenames for the datasets (can verify with training notebook that all sets are the same and that test data wasn't used for training)

In [10]:
#print(filenames_train_wv)

In [11]:
#print(filenames_val_wv)

In [12]:
#print(filenames_test_wv)

## 3. Testing

### Test the baseline SVM

In [36]:
def get_accuracy_svm(model, X, y):
    predictions = model.predict(X)
    num_correct = np.sum((predictions - y) == 0)
    n = X.shape[0]
    accuracy = num_correct/n
    print("The number of correct predictions is: ", num_correct)
    print("The number of files is: ", n)
    print("The accuracy is: ", accuracy)

In [37]:
#Flatten channels to single vector for SVM
X_train_svm = X_train.reshape(X_train.shape[0], 4*(256*5))
X_val_svm = X_val.reshape(X_val.shape[0], 4*(256*5))
X_test_svm = X_test.reshape(X_test.shape[0], 4*(256*5))

In [38]:
filename = 'Trained Model Weights/svm_kernel-sigmoid_C-100_5s.sav'
model = pickle.load(open(filename, 'rb'))

TypeError: ignored

In [ ]:
print("Training Accuracy")
get_accuracy_svm(model, X_train_svm, y_train)
print("\nValidation Accuracy")
get_accuracy_svm(model, X_val_svm, y_val)

In [ ]:
print("Test Accuracy")
get_accuracy_svm(model, X_test_svm, y_test)

### Test the ANN models

In [17]:
def createTensorDataset(data, labels):
    # Returns a tensor dataset object given np arrays of the data and labels
    data_tensor = torch.tensor(data)
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(data_tensor, labels_tensor)
    return dataset

def get_accuracy(model, data_loader, train=False, use_cuda=True):
    correct = 0
    total = 0
    for file, labels in iter(data_loader):
        file = file.float()
        labels = labels.long()
        if use_cuda and torch.cuda.is_available():
            file = file.cuda()
            labels = labels.cuda()
        output = model(file)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += file.shape[0]
    return correct / total

def print_model_predictions(model, data_loader, filenames_list, use_cuda=True):
    #Use a batch size of 1 with shuffle=False for the data_loader passed in
    num_correct = 0
    total = 0
    for file, labels in iter(data_loader):
        file = file.float()
        labels = labels.long()
        if use_cuda and torch.cuda.is_available():
            file = file.cuda()
            labels = labels.cuda()
        output = model(file)
        pred = output.max(1, keepdim=True)[1]
        if pred.item() == labels.item():
            status = "Prediction Correct"
            num_correct += 1
        else:
            status = "Prediction Incorrect"
        total += 1
        print("Prediction: ", pred.item(), " | Label: ", labels.item(), " | Status: ", status, " | File: ", filenames_list[total-1])
    print("Files classified correctly: ", num_correct)
    print("Total files: ", total)
    print("Accuracy: ", num_correct/total)

In [18]:
class ANN_TS_2L(nn.Module):
    def __init__(self):
        super(ANN_TS_2L, self).__init__()
        self.name = "ANN_TS_2L"
        self.layer1 = nn.Linear((256*5)*4, 200)
        self.layer2 = nn.Linear(200, 50)
        self.layer3 = nn.Linear(50, 2)
    def forward(self, img):
        flattened = img.view(-1, (256*5)*4)
        activation1 = F.relu(self.layer1(flattened))
        activation2 = F.relu(self.layer2(activation1))
        output = self.layer3(activation2)
        return output

class ANN_TS_3L(nn.Module):
    def __init__(self):
        super(ANN_TS_3L, self).__init__()
        self.name = "ANN_TS_3L"
        self.layer1 = nn.Linear((256*5)*4, 500)
        self.layer2 = nn.Linear(500, 200)
        self.layer3 = nn.Linear(200, 50)
        self.layer4 = nn.Linear(50, 2)
    def forward(self, img):
        flattened = img.view(-1, (256*5)*4)
        activation1 = F.relu(self.layer1(flattened))
        activation2 = F.relu(self.layer2(activation1))
        activation3 = F.relu(self.layer3(activation2))
        output = self.layer4(activation3)
        return output

#### 2 Layer ANN on 5s Recordings

##### Trained with batch_size=50

In [19]:
pth = 'Trained Model Weights/ANN_TS_2L Weights 2021-03-14 20-28/model_ANN_TS_2L_bs50_lr0-01_epoch67_iteration475_val-acc62-6667.pth'
model_2L = ANN_TS_2L()
model_2L.load_state_dict(torch.load(pth))
model_2L.eval()

ANN_TS_2L(
  (layer1): Linear(in_features=5120, out_features=200, bias=True)
  (layer2): Linear(in_features=200, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=2, bias=True)
)

In [20]:
batch_size = 1

train_dataset = createTensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

val_dataset = createTensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = createTensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
print("Training Set Accuracy")
get_accuracy(model_2L, train_loader)

Training Set Accuracy


0.8314285714285714

In [22]:
print("Validation Set Accuracy")
get_accuracy(model_2L, val_loader)

Validation Set Accuracy


0.8133333333333334

In [23]:
print("Test Set Accuracy")
get_accuracy(model_2L, test_loader)

Test Set Accuracy


0.9066666666666666

In [24]:
print_model_predictions(model_2L, test_loader, filenames_test)

Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T17 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T15 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T39 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T35 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T21 Open (valve) - Filtered
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T11 Open (valve) - Filtered
Prediction:  1  | Label: 

##### Trained with batch_size=75

In [25]:
pth = 'Trained Model Weights/ANN_TS_2L Weights 2021-03-14 20-46/model_ANN_TS_2L_bs75_lr0-01_epoch93_iteration466_val-acc62-6667.pth'
model_2L_bs75 = ANN_TS_2L()
model_2L_bs75.load_state_dict(torch.load(pth))
model_2L_bs75.eval()

ANN_TS_2L(
  (layer1): Linear(in_features=5120, out_features=200, bias=True)
  (layer2): Linear(in_features=200, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=2, bias=True)
)

In [26]:
print("Training Set Accuracy")
get_accuracy(model_2L_bs75, train_loader)

Training Set Accuracy


0.8314285714285714

In [27]:
print("Validation Set Accuracy")
get_accuracy(model_2L_bs75, val_loader)

Validation Set Accuracy


0.7866666666666666

In [28]:
print("Testing Set Accuracy")
get_accuracy(model_2L_bs75, test_loader)

Testing Set Accuracy


0.84

In [29]:
print_model_predictions(model_2L_bs75, test_loader, filenames_test)

Prediction:  0  | Label:  0  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T18 Close (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T6 Open (valve) - Filtered
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T1 Open (valve)_(1) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T33 Open (valve)_(1) - Filtered
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ2_T4 Close (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T37 Open (valve)_(1) - Filtered
Predictio

#### 3 Layer ANN on 5s Recordings

In [29]:
pth = 'Trained Model Weights/ANN_TS_3L Weights 2021-03-14 20-38/model_ANN_TS_3L_bs50_lr0-001_epoch72_iteration509_val-acc65-3333.pth'
model_3L = ANN_TS_3L()
model_3L.load_state_dict(torch.load(pth))
model_3L.eval()

ANN_TS_3L(
  (layer1): Linear(in_features=5120, out_features=500, bias=True)
  (layer2): Linear(in_features=500, out_features=200, bias=True)
  (layer3): Linear(in_features=200, out_features=50, bias=True)
  (layer4): Linear(in_features=50, out_features=2, bias=True)
)

In [30]:
print("Training Set Accuracy")
get_accuracy(model_3L, train_loader)

Training Set Accuracy


0.8542857142857143

In [31]:
print_model_predictions(model_3L, train_loader, filenames_train)

Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T5 Open (valve)_(1) - Filtered
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect  | File:  /content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T39 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T35 Open (valve)_(2) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T75 Open (valve) - Filtered
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ2_T1 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW_T27 Open (valve) - Filtered
Prediction:  1  |

In [32]:
print("Validation Set Accuracy")
get_accuracy(model_3L, val_loader)

Validation Set Accuracy


0.8266666666666667

In [33]:
print_model_predictions(model_3L, val_loader, filenames_val)

Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T15 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T17 Open (valve)_(2) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T34 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T3 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T30 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T33 Open (valve) - Filtered
Prediction:  1  | Label: 

In [34]:
print("Test Set Accuracy")
get_accuracy(model_3L, test_loader)

Test Set Accuracy


0.8666666666666667

In [35]:
print_model_predictions(model_3L, test_loader, filenames_test)

Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T17 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T15 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T39 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T35 Open (valve) - Filtered
Prediction:  1  | Label:  1  | Status:  Prediction Correct  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T21 Open (valve) - Filtered
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect  | File:  /content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T11 Open (valve) - Filtered
Prediction:  1  | Label: 

### Test the CNN model

In [55]:
# Upload the file manually since it is too large to store on GitHub
#from google.colab import files
#uploaded = files.upload()

In [56]:
def downsampleTensorHW(dataset, factor):
    # Given a tensor dataset of data, labels, takes data of shape [N, C, H, W]
    # Returns the dataset, but with data of shape [N, C, H/factor, W/factor]
    data = dataset.tensors[0]
    labels = dataset.tensors[1]
    data_new = F.interpolate(data, size=(int(data.shape[-2] /factor), int(data.shape[-1] /factor)), mode='bicubic', align_corners=False)
    return TensorDataset(data_new, labels)

In [57]:
class CNN_WV(nn.Module):
    def __init__(self):
        super(CNN_WV, self).__init__()
        self.name = "CNN_WV"
        
        self.conv1 = nn.Conv2d(4, 8, 4) #in_channels, out_channels (# of kernels to try), kernel_size. Size in is 4x30x640
        self.pool = nn.MaxPool2d(2, 2) #kernel_size (for the pool), stride. Size in should be 8x26x636 
        self.conv2 = nn.Conv2d(8, 16, 4) #size in should be 8x13x318
        self.pool = nn.MaxPool2d(2, 2) # size in should be 16x9x314
        
        self.fc1 = nn.Linear(16*5*157, 2500) #size in should be 16x5x157
        self.fc2 = nn.Linear(2500, 500)
        self.fc3 = nn.Linear(500, 50)
        self.fc4 = nn.Linear(50, 2)
 
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1) #flatten the input

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x)) 
        x = self.fc4(x)
        x = x.squeeze(1)
        return x

In [58]:
#pth = 'model_CNN_WV_bs20_lr0-00025_epoch3_iteration58_val-acc64-0.pth'
#model_CNN = CNN_WV()
#model_CNN.load_state_dict(torch.load(pth))
#model_CNN.eval()

In [59]:
#batch_size = 1

#train_dataset = createTensorDataset(X_train_wv, y_train_wv)
#train_dataset = downsampleTensorHW(train_dataset, 2)
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

#val_dataset = createTensorDataset(X_val_wv, y_val_wv)
#val_dataset = downsampleTensorHW(val_dataset, 2)
#val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#test_dataset = createTensorDataset(X_test_wv, y_test_wv)
#test_dataset = downsampleTensorHW(test_dataset, 2)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [60]:
#print("Training Set Accuracy")
#get_accuracy(model_CNN, train_loader)

In [61]:
#print("Validation Set Accuracy")
#get_accuracy(model_CNN, val_loader)

In [62]:
#print("Test Set Accuracy")
#get_accuracy(model_CNN, test_loader)

In [63]:
#print_model_predictions(model_CNN, test_loader, filenames_test)

## 4. Discussion

The table below summarizes the accuracy of each model on the training, validation, and testing sets:

|                          |Training |Validation|Testing  |       
|--------------------------|:-------:|:--------:|:-------:|
|Baseline SVM              |68.6%    |77.3%     |65.3%    |
|2 Layer ANN, 50 batch size|83.1%    |81.3%     |90.7%    |
|2 Layer ANN, 75 batch size|83.1%    |78.7%     |  84.0%  |
|3 Layer ANN               |**85.4%**|**82.7%** |**86.7%**|
|CNN                       |66.3%    |56.0%     |60.0%    |

The final model is chosen to be the 3 Layer ANN since it had the highest training and validation accuracies. Although the 2 layer ANN trained with a batch size of 50 ended up having a higher performance on the test set, there would have been no way to know this a priori, and selecting this model as the final model would mean that testing data would be influencing model selection, which contradicts the purpose of having a held-out testing set.

The baseline SVM model, which had accuracies in the mid 60s, is similar to the results obtained by [Sereshkeh et al.](https://www.worldscientific.com/doi/abs/10.1142/S0129065717500332), who also built a SVM classifier for covert speech, but of "yes" and "no" rather than "open" and "close". Their model had an accuracy of 69%. However, the EEG signals in their model had been processed much more than ours; in addition to applying a 0.5-50Hz bandpass like we did, they also used the [ADJUST algorithm](https://www.nitrc.org/projects/adjust/#:~:text=ADJUST%20is%20a%20completely%20automatic,isolating%20artifacts%20from%20EEG%20recordings.) to remove blinks, and then took a wavelet transform. This is also where our results differed; when we took wavelet transforms and built a CNN based on the data, accuracies were lower than any other model. We believe this may have occured for two reasons:

> 1) The wavelet transform inputs were much larger data files, so passing through the network made training time-consuming. Thus, it is possible to have undertrained.
>
> 2) The wavelet transform, but more generally frequency domain components, may not be of much help in covert speech classification. In our preliminary model training, we noticed that using a signal's FFT rather than its time series provided poorer results when using an ANN classifier. With wavelets, the increased resolution in the frequency domain, but lower resolution in the time domain may have led to obfuscating classification features and increasing noise.

Comparing the ANN models, results are largely similar, but the 3 layer model has slightly higher training and validation accuracies compared to the 2 layer models. By having an extra layer, the chosen model may be better at modelling nonlinearities in the EEG signals; in the long run and with more testing data, we would expect the 3 layer model to perform better than the 2 layer models.